In [2]:
import json

import subprocess
import time

from os import path

In [13]:
metadata_repo = "/home/aaron/workspace/camhd_analysis/CamHD_motion_metadata"
movie_repo = "/home/aaron/canine/workspace/camhd_analysis/test_data"

movie = "/RS03ASHS/PN03B/06-CAMHDA301/2016/01/01/CAMHDA301-20160101T000000Z.mov"

regions_filename = metadata_repo + (path.splitext(movie)[0]) + "_optical_flow_regions.json"
movie_filename = movie_repo + movie

out_file = path.splitext(path.split(movie)[1])[0] + "_subtitles.mkv"

In [14]:
with open(regions_filename,'r') as infile:
    j = json.load( infile )
   
regions = j['regions']

In [21]:
with open('subtitles.srt','w') as srtfile:

    index = 1
    for region in regions:

        bounds = region['bounds']

        #preseek_time = time.strftime("%H:%M:%S", time.gmtime( r[0]/29.97-2 ) )
        start_time = time.strftime("%H:%M:%S", time.gmtime( bounds[0]/29.97 ) )
        end_time = time.strftime("%H:%M:%S", time.gmtime( bounds[1]/29.97 ) )
        
        duration = time.strftime("%H:%M:%S", time.gmtime( (bounds[1] - bounds[0])/29.97 ) )

        srtfile.write("%d\n" % index)
        
        ## Fake sub-second timestamps for now...
        srtfile.write("%s,000 --> %s,000\n" % (start_time, end_time) )
        srtfile.write("%s\n" % region["type"])
        srtfile.write("\n")
        
        index += 1


In [18]:
## N.b.  The SRT file format is pretty delicate.  If ffmpeg won't run, the .srt file may be malformed...

subprocess.run( ['ffmpeg', '-y', '-i', movie_filename, '-vf', 'subtitles=subtitles.srt', out_file])

CompletedProcess(args=['ffmpeg', '-y', '-i', '/home/aaron/canine/workspace/camhd_analysis/test_data/RS03ASHS/PN03B/06-CAMHDA301/2016/01/01/CAMHDA301-20160101T000000Z.mov', '-vf', 'subtitles=./subtitles.srt', 'CAMHDA301-20160101T000000Z_subtitles.mkv'], returncode=1)